### Calculate the gold table from the silver table 
##### Use Latest Inventory Snapshot with the Inventory Change to calculate the Current Inventory 



In [0]:
import dlt as dlt
from pyspark.sql.functions import *


In [0]:
@dlt.table(
  name='current_inventory',
  comment='current inventory count for a product in a store location',
  table_properties={'quality':'gold'},
  spark_conf={'pipelines.trigger.interval': '5 minutes'}
  )
def current_inventory():

    # calculate inventory change with bopis corrections
    inventory_change_df = (
        dlt
        .read('inventory_change').alias('x')
        .join(
            dlt.read('Store').alias('y'), 
            on='store_id'
            )
        .join(
            dlt.read('inventory_change_type').alias('z'), 
            on='change_type_id'
            )
        .filter(expr("NOT(y.name='online' AND z.change_type='bopis')"))
        .select('store_id','item_id','date_time','quantity')
        )

    # calculate current inventory
    inventory_current_df = (
        dlt
            .read('latestinventorysnapshot').alias('a')
            .join(
            inventory_change_df.alias('b'), 
            on=expr('''
                a.store_id=b.store_id AND 
                a.item_id=b.item_id AND 
                a.date_time<=b.date_time
                '''), 
            how='leftouter'
            )
            .groupBy('a.store_id','a.item_id')
            .agg(
                first('a.quantity').alias('snapshot_quantity'),
                sum('b.quantity').alias('change_quantity'),
                first('a.date_time').alias('snapshot_datetime'),
                max('b.date_time').alias('change_datetime')
                )
            .withColumn('change_quantity', coalesce('change_quantity', lit(0)))
            .withColumn('current_quantity', expr('snapshot_quantity + change_quantity'))
            .withColumn('date_time',expr('GREATEST(snapshot_datetime, change_datetime)'))
            .drop('snapshot_datetime','change_datetime')
            .orderBy('current_quantity')
            )

    return inventory_current_df